In [1]:
from pinecone import Pinecone as PineconeClient
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec
from dotenv import dotenv_values
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
#from langchain_community.embeddings import CohereEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Pinecone 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
import requests
import json
import ray

In [3]:
config = dotenv_values(".env")
env_key = config["PINE_CONE_ENV_KEY"]
api_key = config["PINE_CONE_API_KEY"]
openai_api_key=config["OPENAI_API_KEY"]
cohere_api_key = config["COHERE_API_KEY"]
#pc_index = config["INDEX_NAME"]

In [4]:
pineCone = PineconeClient(
    api_key=api_key
)

In [6]:
# create the cohere embeddings
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

pc = PineconeGRPC()
index_name = 'cohere-wikipedia'
embeddings = CohereEmbeddings(model="multilingual-22-12", cohere_api_key=cohere_api_key)
vectorstore = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [7]:
# fetch wiki using id
def fetch_wiki_page(id):
    # expand the context with which we search for chunks outside of LLM
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"
    response = requests.get(url)
    data = response.json()
    page_content = list(data['query']['pages'].values())[0]['extract']
    return page_content

# fetch the 
def fetch_url(x):
    urls = [doc.metadata['url'] for doc in x['context']]
    ids = [url.split('=')[-1] for url in urls]
    contents = [fetch_wiki_page(id)[:32000] for id in ids]
    return {"context": contents, "question": x["question"]}

In [15]:
# RAG Prompt
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.load.dump import dumps
from langsmith import traceable

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG Model
#    | RunnableLambda(fetch_url)
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview", openai_api_key=openai_api_key)

# create an autotrace for the pipeline
# TODO: Note we do not need traceable here as we've created a new LangSmith project
# to log all of our LangChain projects
# @traceable
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | RunnableLambda(fetch_url) # pipe the entire context from the url
    | prompt
    | model
    | StrOutputParser()
)

chain_result = chain.invoke("What is film noir?")

# Invoke a question to the chain
#    SystemMessage(content="You're a great assistant"),
# message = HumanMessage(content="What is film noir?")
# chain_result = chain.invoke("what is film noir?")
# chain_result = chain.invoke("what is film noir?")
messages = [
    SystemMessage(content="You are Micheal Jordan."),
    HumanMessage(content="Which shoe manufacturer are you associated with?"),
]
# chain_result = chain.invoke(messages)
#chain_result = chain.invoke("What is film noir?")
# chain_result = chain.run("What is film noir?")
# chain_result = pipeline("What is film noir?")
print("Chain result output:")
print(chain_result)

Chain result output:
Film noir is a cinematic term used primarily to describe stylized Hollywood crime dramas, particularly those that emphasize cynical attitudes and motivations. The classic period of American film noir is generally regarded as the 1940s and 1950s. This era of film noir is associated with a low-key, black-and-white visual style that has roots in German Expressionist cinematography. The stories and attitudes of classic noir often derive from the hardboiled school of crime fiction that emerged in the United States during the Great Depression.
